# Lesson 2 - Calculating the Average Clutter Height using Lidar

In this lesson you will determine the average height of the clutter of the Martin Acres neighborhood in the city of Boulder, Colorado. 

<img src="./images/martin_acres_labeled.png" alt="Martin Acres Map" width="800"/>

The image shows a map of Boulder, Colorado with Martin Acres circled. Martin Acres is a residential neighborhood containing mostly single-family homes. You will use lidar data to determine the average height of the clutter (buildings and trees) in this region. 

The following cell will import the needed Python libraries.

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

### Lidar Data Files

The lidar data files for this tutorial are in the `course-materials/data/` directory. 

- **`course-materials/data/MartinAcres.dtm.tif`**
- **`course-materials/data/MartinAcres.dsm.tif`**

For Martin Acres there are two `.tif` files: a Digital Surface Model (DSM), and a Digital Terrain Model (DTM). The DSM contains the elevation of the tallest points in the area, which is often the tops of trees and buildings. The DTM contains the elevation of the ground. 

Each `.tif` file is a [GeoTIFF](https://www.ogc.org/standard/geotiff/) file, which adds georeferenced metadata to the image data. Each pixel in each band of the raster image represents a data value referenced to a specific geographic location. The file's metadata includes all the information about the coordinate system, map projection, and units that are needed to spatially reference the elevation data.

<img src="./images/DSM_DTMcropped.png" alt="DSM DTM" width="500"/>

Image from ["What is a CHM, DSM and DTM? About Gridded, Raster LiDAR Data" by Leah A. Wasser](https://www.neonscience.org/resources/learning-hub/tutorials/chm-dsm-dtm).



The following cell opens and reads the DTM and DSM files. It also sets variables with the boundary coordinates of the `.tif` files.

In [ ]:
## get Digital Terrain Model
dataset_terrain = rasterio.open("./data/MartinAcres.dtm.tif")
## get Digital Surface Model
dataset_surface = rasterio.open("./data/MartinAcres.dsm.tif")

## read the elevation layer
band1_terrain = dataset_terrain.read(1)
band1_surface = dataset_surface.read(1)

## dataset_terrain and dataset_surface uses EPSG:32613 coordinate system, unit is in meters
## print the bounds of the dataset
print("The bounds of the lidar dataset. In meters:\n ", dataset_terrain.bounds) 

## lidar boundries in meters (EPSG:32613)
left, bottom, right, top = dataset_terrain.bounds

## lidar boundries in Lat, Long coordinates
## EPSG 32613 converted to EPSG:4326 (used in GPS) with https://epsg.io/transform#s_srs=32613&t_srs=4326&x=NaN&y=NaN
left_deg, bottom_deg, right_deg, top_deg = (-105.2869255, 39.9747967, -105.2343079, 40.0019534)
print("The bounds of the lidar dataset. In lat/long coordinates:\n ", left_deg, bottom_deg, right_deg, top_deg) 

## For converting lat and long degrees into meters (only used in small distances)
meters_per_lat = 111319.49 
meters_per_long = 85263.24 ## only valid at latitudes of 40 degrees

### Visualize MartinAcres.dtm.tif and MartinAcres.dsm.tif

Now that we've loaded the raster data for Martin Acres, let's visualize the terrain and surface elevations in the area. We can do this by passing the raster data to Matplotlib, the same plotting library we've already used.

The visualization of the digital terrain model shows large-scale terrain variation from the Martin Acres area into the foothills of the Rocky Mountains. The visualization of the digital surface model further resolves surface structures, such that features like roads and buildings become visible on top of the terrain variations.

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Elevation Heatmap of MartinAcres.dtm.tif (Terrain)")
plt.xlabel("Meters")
plt.ylabel("Meters")
dtm_plt = plt.imshow(band1_terrain, cmap='pink')
cbar = plt.colorbar(dtm_plt, fraction=0.03)
cbar.set_label('Elevation (meters)')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Elevation Heatmap of MartinAcres.dsm.tif (Surface)")
plt.xlabel("Meters")
plt.ylabel("Meters")
dtm_plt = plt.imshow(band1_surface, cmap='pink')
cbar = plt.colorbar(dtm_plt, fraction=0.03)
cbar.set_label('Elevation (meters)')
plt.show()

### Find the _Ground_ and _Clutter_ elevations at the center of Martin Acres

To do this, we define two helpful functions. The first, `convert_gps_to_meters`, produces coordinate values referenced to the raster data origin point, in meters, from a latitude/longitude pair. Next, the `get_elev` function takes a latitude and longitude coordinate pair, as well as raster data, and retrieves the elevation at the specified location from the provided raster data (terrain or surface).

In [ ]:
def convert_gps_to_meters(lat: float, long: float):
    """Convert a (lat, lon) coordinate pair to meters-based coordinate pair"""
    # top_deg and left_deg are raster data boundaries defined above
    lat_dif = top_deg - lat
    long_dif = long - left_deg
    # Get meters-based coordinates, referenced to raster data origin
    vert_dif_m = lat_dif * meters_per_lat
    horz_dif_m = long_dif * meters_per_long
    return (vert_dif_m, horz_dif_m)


def get_elev(lat: float, long: float, lidar_model: rasterio.io.DatasetReader, lidar_elev_band: np.ndarray):
    """Get the elevation from the specified lidar dataset for a given (lat, lon) coordinate
    
    :param lat: Latitude of the desired elevation data, in degrees
    :param long: Longitude of the desired elevation data, in degrees
    :param lidar_model: Lidar dataset loaded with rasterio.open()
    :param lidar_elev_band: Lidar data array, loaded with rasterio.io.DatasetReader.read()
    """
    vert_dif_m, horz_dif_m = convert_gps_to_meters(lat, long)
    row, col = lidar_model.index(left + horz_dif_m, top - vert_dif_m) ## get the row and colomn where the elevation is stored
    elev = lidar_elev_band[row][col] ## retrieve the elevation
    return elev

Now we can use these functions to retrieve elevation data at a point we choose:

__Martin Acres center__: <br/>
Latitude = 39.9916 <br/>
Longitude = -105.2510

We can also define and calculate the "clutter height" as the difference between the surface and terrain elevations at a given location. 

In [ ]:
## set Martin Acres center location
ma_center = (39.9916, -105.2510)
print(f"Martin Acres center {ma_center}:")

## find the terrain elevation
terrain_elev = get_elev(ma_center[0], ma_center[1], dataset_terrain, band1_terrain)
print(f"The ground elevation is {terrain_elev:.1f} meters")

## find the surface elevation
surface_elev = get_elev(ma_center[0], ma_center[1], dataset_surface, band1_surface)
print(f"The surface elevation is {surface_elev:.1f} meters")

## find the clutter height above the surface
clutter_height = surface_elev - terrain_elev
print(f"The clutter height is {clutter_height:.2f} meters")

### Take Clutter Height Samples

Let's get more clutter height data in the Martin Acres neighborhood. Below, we generate 1000 random locations within 500 meters of the chosen "center point" of the Martin Acres neighborhood, then find the clutter height at each point. From here onwards, we will only consider a given location to have clutter if the calculated "clutter height" exceeds 2 meters. Finally, we will plot the results on a map.

In [ ]:
## convert chosen center location from lat/long to meters
ma_center_meters = convert_gps_to_meters(ma_center[0], ma_center[1])

print(f"Center Point is at (x, y) = ({ma_center_meters[1]:.2f}, {ma_center_meters[0]:.2f}) (meters).")
print("The origin is the top left corner of the map.")


In [ ]:
## generate 1000 random x,y locations within 1 km the Martin Acres center
num_rands = 1000
radius = 500  # meters
x_random = np.random.randint(ma_center_meters[1]-radius, ma_center_meters[1]+radius, num_rands)
y_random = np.random.randint(ma_center_meters[0]-radius, ma_center_meters[0]+radius, num_rands)

## create lists to hold the locations and clutter heights that are within 500 meters of the Example Point
within_500m_x = []
within_500m_y = []
clutter_height_ls = []

## loop through all of the random points
for i in range(len(x_random)):
    ## find distance from center point to a random point
    distance = np.sqrt(np.square(ma_center_meters[1]-x_random[i]) + np.square(ma_center_meters[0]-y_random[i]))
    
    ## if distance is within the specified radius, find the clutter height at that location
    if distance < radius:
        ground_elev = band1_terrain[y_random[i]][x_random[i]]
        clutter_elev = band1_surface[y_random[i]][x_random[i]]
        clutter_height = clutter_elev - ground_elev
        ## "clutter height" must be higher than 2 meters to be considered
        if clutter_height > 2:
            clutter_height_ls.append(clutter_height)
            within_500m_x.append(x_random[i])
            within_500m_y.append(y_random[i])

## Plotting

## read in Martin Acres map image
##  image from www.openstreetmap.org
martin_acres_map = plt.imread('./images/martin_acres_map.png')

## set the map bounding box (in meters)
BBox = (0,  right-left,      
        top-bottom,  0)

fig, ax = plt.subplots(figsize=(10,10))

# Plot base map
ax.imshow(martin_acres_map, zorder=0, extent = BBox, aspect= 'equal')

# Plot randomly-sampled points with color indicating clutter height
ex1 = ax.scatter(within_500m_x, within_500m_y, zorder=1, c=clutter_height_ls, cmap='Blues', alpha=1.0, s=25)
plt.colorbar(ex1, fraction=0.03, label="Clutter Height (meters)")

# Plot chosen centerpoint of Martin Acres
ax.scatter(*ma_center_meters[::-1], label="Martin Acres Center", marker="^", c="r", s=100, edgecolors="k")

ax.set_title('Randomly Sampled Clutter Heights\n within 500 meters of the center')
ax.set_xlabel("Meters")
ax.set_ylabel("Meters")
ax.set_xlim(BBox[0]+(BBox[1]/3), BBox[1])
ax.set_ylim(BBox[2]-(BBox[2]/3), BBox[3])
ax.legend()
plt.show()


### Do Clutter Height Sampling With A Lot More Random Locations (~500,000 Samples)

Same code as previous cell, starting with 1 million random locations before narrowing down to points:
  * Within 500 meters of Martin Acres center.
  * Having at least 2 meters of clutter height.

In [ ]:
num_rands = 1000000
radius = 500

## generate random x,y locations
x_random = np.random.randint(ma_center_meters[1]-radius, ma_center_meters[1]+radius, num_rands)
y_random = np.random.randint(ma_center_meters[0]-radius, ma_center_meters[0]+radius, num_rands)

## create lists to hold the locations and clutter heights that are within 500 meters of the center
within_500m_x = []
within_500m_y = []
clutter_height_ls = []

## loop through all of the random points
for i in range(len(x_random)):
    ## print the progress
    if i % 10000 == 0 or i == len(x_random) - 1:
        print(f"\rProgress {i+1}/{num_rands}", end='')
        
    ## find distance from Example Point to a random point
    distance = np.sqrt(np.square(ma_center_meters[1]-x_random[i]) + np.square(ma_center_meters[0]-y_random[i]))
    
    ## if distance is within 500 meters, find the clutter height at that location
    if distance < radius:
        ground_elev = band1_terrain[y_random[i]][x_random[i]]
        clutter_elev = band1_surface[y_random[i]][x_random[i]]
        clutter_height = clutter_elev - ground_elev

        ## "clutter height" must be higher than 2 meters to be considered
        if clutter_height > 2:
            clutter_height_ls.append(clutter_height)
            within_500m_x.append(x_random[i])
            within_500m_y.append(y_random[i])

print(f"\n{len(clutter_height_ls)} successful clutter samples")

### Plot a Histogram of the Clutter Heights Sampled in Martin Acres

We now have hundreds of thousands of clutter height datapoints within the analysis region, a circle of radius 500 meters surrounding our chosen center point. Let's view the distribution of clutter heights within this area.

In [ ]:
# the histogram of the data
fig, ax = plt.subplots(figsize=(10, 5))
n, bins, patches = ax.hist(clutter_height_ls, np.linspace(2,35,34), density=True, edgecolor="k", facecolor='g')

ax.set_xlabel('Clutter Height (meters)')
ax.set_ylabel('Probability')
ax.set_title('Histogram of Clutter Heights of Martin Acres')
ax.grid(True, which="both", axis="both", alpha=0.3)
ax.minorticks_on()
ax.set_axisbelow(True)
plt.show()

The most common clutter height in this region is between 3 to 5 meters. Print the mean, median, and standard deviation of the Martin Acres clutter.

In [ ]:
mean_clutter_height = np.mean(clutter_height_ls)
median_clutter_height = np.median(clutter_height_ls)
std_clutter_height = np.std(clutter_height_ls)
print("Martin Acres Clutter Heights:")
print(f" Mean = {mean_clutter_height:.1f} meters")
print(f" Median = {median_clutter_height:.1f} meters")
print(f" Standard Deviation = {std_clutter_height:.1f} meters")

Well done, you've found clutter statistics for the Martin Acres neighborhood using lidar data! In the next lesson you'll use these statistics to inform a clutter model.

End of Lesson 2.